In [ ]:
import pickle
from pathlib import Path
import pandas as pd
import numpy as np
import torch.nn as nn

from neuralhydrology.utils.config import Config

import matplotlib.pyplot as plt
import torch
from neuralhydrology.evaluation import metrics
from neuralhydrology.nh_run import start_run, eval_run

import os

from neuralhydrology.modelzoo.cfe_modules.cfe_dataclasses import (
    GroundwaterStates,
    RoutingInfo,
    SoilConfig,
    SoilStates,
    get_constants,
)

from neuralhydrology.modelzoo.cfe_modules.get_default_params import get_default_params

# Validate Physics of CFE to C Code
The goal of this section is to compare the existing dCFE code (as of 20241215) to CFE hourly outputs: https://github.com/NWC-CUAHSI-Summer-Institute/cfe_py/blob/main/cat58_test_compare.csv The forcing for the same time period is: https://github.com/NWC-CUAHSI-Summer-Institute/cfe_py/blob/main/cat58_01Dec2015.csv The basin settings (since baasin id unknown) is here: https://github.com/NWC-CUAHSI-Summer-Institute/cfe_py/blob/main/bmi_cfe.py

Note that the forcing file as TMP_2maboveground which I assume is temperature in (K), so it needs to be converted to C before being used. And the initialized_basin_constants is adaped based on the bmi_cfe.py file.

In [ ]:
# Set test data directory to NEURALHYDROLOGY-DCFE/test/test_data/cfe_data
test_data_dir = Path("/Users/ziyu/Library/CloudStorage/OneDrive-ColoradoSchoolofMines/Documents/College/ResearchStuff/NextGen/neuralhydrology-dcfe/test/test_data/cfe_data")

device = 'cpu'
batch_size = 2

forcingFile = pd.read_csv(test_data_dir/'cat58_01Dec2015.csv')
forcingsTest = torch.zeros(batch_size, forcingFile.shape[0], 3) #[ "batch_size", time_step, num_forcings]

forcingsTest[:, :, 0] = (torch.tensor(forcingFile.loc[:,'precip_rate']) * 1000 * 3600) # m/s to mm/hr
forcingsTest[:, :, 1] = (torch.tensor(forcingFile.loc[:,'TMP_2maboveground']) - 273.15) # K to C
forcingsTest[:, :, 2] = torch.tensor(forcingFile.loc[:,'DSWRF_surface']) # same unit as camels

# empty vector to store output
Discharge = torch.zeros(forcingsTest.shape[1], 8)

cfe_features = {
    'catchment_area_km2': 15.617167355002097 * torch.tensor(1.0, dtype=torch.float32, device=device).repeat(batch_size), 
    'refkdt': 3.8266861353378374 * torch.tensor(1.0, dtype=torch.float32, device=device).repeat(batch_size),
    'max_gw_storage': 16 * torch.tensor(1.0, dtype=torch.float32, device=device).repeat(batch_size),
    'Cgw': 0.01*torch.tensor(1.0).repeat(batch_size),
    'expon': 6.0 * torch.tensor(1.0, dtype=torch.float32, device=device).repeat(batch_size),
    #gw_storage = 0.05 * torch.ones((  batch_size,  shape[1]), dtype=torch.float32, device= device)
    'alpha_fc': 0.33 * torch.tensor(1.0, dtype=torch.float32, device=device).repeat(batch_size ),
    'K_nash': 0.03 * torch.tensor(1.0, dtype=torch.float32, device= device).repeat(  batch_size), 
    'K_lf': 0.01 * torch.tensor(1.0, dtype=torch.float32, device= device).repeat(  batch_size), 
    'nash_storage': torch.zeros((  batch_size,2), dtype=torch.float32, device= device),
    'giuh_ordinates': torch.tensor([0.1, 0.35, 0.2, 0.14, 0.1, 0.06, 0.05], dtype=torch.float32, device= device),
    'depth': 2.0 * torch.tensor(1.0, dtype=torch.float32, device= device).repeat(  batch_size), # not sure where they got these values, they don't match CAMELS, [m]
    'bb': 4.05 * torch.tensor(1.0, dtype=torch.float32, device= device).repeat(  batch_size), # exponent on Clapp-Hornberger function, part of calibration
    'satdk': 0.00000338*torch.tensor(1.0).repeat(  batch_size),
    'satpsi': 0.355 * torch.tensor(1.0, dtype=torch.float32, device= device).repeat(  batch_size), 
    'slop': 1 * torch.tensor(1.0, dtype=torch.float32, device= device).repeat(  batch_size), # slope coefficient, part of calibration
    'smcmax': 0.439 * torch.tensor(1.0, dtype=torch.float32, device= device).repeat(  batch_size), # maximum soil moisture content [m3/m3], part of calibration
    'wltsmc': 0.066* torch.tensor(1.0, dtype=torch.float32, device= device).repeat(  batch_size),
    'D': 2.0 * torch.tensor(1.0, dtype=torch.float32, device= device).repeat(  batch_size),
    'mult': 1000.0 * torch.tensor(1.0, dtype=torch.float32, device= device).repeat(  batch_size)
}

# Grab config file that trained this model
config_path = Path('/Users/ziyu/Library/CloudStorage/OneDrive-ColoradoSchoolofMines/Documents/College/ResearchStuff/NextGen/neuralhydrology-dcfe/examples/07-DCFE/2basinTest_devMultiBasin.yml')
config = Config(config_path, dev_mode=True)

cfe_params = get_default_params(config, cfe_features, device)
constants = get_constants(config.dcfe_hourly)

# initialize model states/reservoirs.
gw_reservoir = GroundwaterStates(device=device, batch_size=batch_size, cfe_params=cfe_params)
soil_config = SoilConfig(cfe_params=cfe_params, device=device, batch_size=batch_size, constants=constants)
soil_reservoir = SoilStates(
    device=device,
    batch_size=batch_size,
    cfe_params=cfe_params,
    soil_config=soil_config,
    constants=constants,
    )
routing_info = RoutingInfo(device=device, batch_size=batch_size, cfe_params=self.cfe_params)